# "Data Cleaning & Pipelines"
> "It's not the sexiest part of data science but it is probably the most important"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [data cleaning, data preparation, pandas.pipe, NBA]
- image: https://media.giphy.com/media/Qvpxb0bju1rEp9Nipy/giphy.gif

## The Jist: Data Cleaning is critical before developing a model
The data exploration post showed how to use knowledge about a dataset to interpret information. Since we know how the 2017-2019 seasons went for the Milwuakee Bucks and Sacramento Kings we can now plan out our machine learning problem. The machine learning model will attempt to predict the outcome of an NBA game before it actually occurs. We can start with using a logistic regression model to get a probabilistic output but we can look into other classification models after we give this one a go. This article outlines the most imperative portion of a machine learning project, outlining the problem and preparing the data. 

### Part 1: Data Exploration 
This post is a continuation of the data exploration post where we explored the 2017-2019 seasons for the Milwuakee Bucks and the Sacramento Kings. Feel free to hop out and pop back in if you want to see the data described and explored: 

[Part 1 post: Data Exploration with NBA Data](https://dpendleton22.github.io/valuebyerror/data%20exploration/box%20plots/histograms/nba/2020/01/14/nba-analysis-post.html)

![KeepItMoving](https://media.giphy.com/media/Ze4BXdrjDjygM9Piq0/giphy.gif)

In [5]:
#hide
import os
from pathlib import Path
import pandas as pd
import numpy as np

### Set all the necessary paths for the data 
The data was provided by https://www.basketball-reference.com/. They are a great source for anyone interested in sports analytics as an initial introduction. I can go into details later within the project to note the importance of detail in sports data.

Using the <i>pathlib</i> library from pandas it's straightforward getting all the data file names set. Setting a base folder name is a good method to simply call each dataset path by their name. Another method to get each dataset path would be to use the <i>glob</i> library to search the dataset folder for files with csv extensions

In [7]:
#collapse-hide
DATA_FOLDER = Path(os.getcwd(), 'mil_sac_data')
sac_2017_szn = Path(DATA_FOLDER, 'sac_2017_2018_szn.csv')
sac_2018_szn = Path(DATA_FOLDER, 'sac_2018_2019_szn.csv')
mil_2017_szn = Path(DATA_FOLDER, 'mil_2017_2018_szn.csv')
mil_2018_szn = Path(DATA_FOLDER, 'mil_2018_2019_szn.csv')

Let's review one of the datasets to determine how they all need to be cleaned

In [23]:
sac_2017_df = pd.read_csv(sac_2017_szn, header=[0,1])

### Hold up, why are you setting the header argument?
Most times than not, calling pd.read_csv("filename") with no additional arguments would read in a dataframe as expected. In this instance, BasketballReference provides two headers in their csv so we need to let pandas know in order to process the dataset. Pandas read_csv() function has over 20 arguments that can be set depending on how the data is parsed and organized in the original file. So if your data is a little funky, the function may still be able to handle it.

Pandas read_csv() documentation: [pandas.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [24]:
#collapse-show
sac_2017_df.iloc[0:5, 0:15]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
                  Rk                  G               Date Unnamed: 3_level_1   
0                  1                  1         2017-10-18                NaN   
1                  2                  2         2017-10-20                  @   
2                  3                  3         2017-10-21                  @   
3                  4                  4         2017-10-23                  @   
4                  5                  5         2017-10-26                NaN   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Unnamed: 7_level_0  \
                 Opp                W/L                 Tm                Opp   
0                HOU                  L                100                105   
1                DAL                  W                 93                 88   
2                DEN                  L                 79                 96   
3                PHO                  L                115                117   
4                NOP                  L                106                114   

  Team                                
    FG FGA    FG%  3P 3PA    3P%  FT  
0   42  88  0.477   8  23  0.348   8  
1   37  87  0.425  10  23  0.435   9  
2   31  85  0.365   8  22  0.364   9  
3   43  99  0.434   9  22  0.409  20  
4   38  81  0.469   7  20  0.350  23

> Tip: Always view the dimensions of your data before analyzing it

In [31]:
print (f"This dataset is {len(sac_2017_df)} in length and contains {len(sac_2017_df.columns)} columns")

This dataset is 82 in length and contains 41 columns


Using df.describe() is an easy and useful way to briefly view the distribution of the dataset across all the columns. This dataset is 82 rows in length which makes sense because there are 82 games in a regular season and contains 41 columns

In [32]:
#collapse-hide
sac_2017_df.iloc[0:5, 0:15].describe()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 6_level_0  \
                      Rk                  G                 Tm   
count           5.000000           5.000000           5.000000   
mean            3.000000           3.000000          98.600000   
std             1.581139           1.581139          13.612494   
min             1.000000           1.000000          79.000000   
25%             2.000000           2.000000          93.000000   
50%             3.000000           3.000000         100.000000   
75%             4.000000           4.000000         106.000000   
max             5.000000           5.000000         115.000000   

      Unnamed: 7_level_0       Team                                  \
                     Opp         FG        FGA       FG%         3P   
count           5.000000   5.000000   5.000000  5.000000   5.000000   
mean          104.000000  38.200000  88.000000  0.434000   8.400000   
std            12.144958   4.764452   6.708204  0.044486   1.140175   
min            88.000000  31.000000  81.000000  0.365000   7.000000   
25%            96.000000  37.000000  85.000000  0.425000   8.000000   
50%           105.000000  38.000000  87.000000  0.434000   8.000000   
75%           114.000000  42.000000  88.000000  0.469000   9.000000   
max           117.000000  43.000000  99.000000  0.477000  10.000000   

                                       
             3PA       3P%         FT  
count   5.000000  5.000000   5.000000  
mean   22.000000  0.381200  13.800000  
std     1.224745  0.038855   7.120393  
min    20.000000  0.348000   8.000000  
25%    22.000000  0.350000   9.000000  
50%    22.000000  0.364000   9.000000  
75%    23.000000  0.409000  20.000000  
max    23.000000  0.435000  23.000000

### Merge multi index headers and remove unwanted tags
In stead of indexing by columns with this notation, 
```python
sac_2017_df[('Unnamed: 0_level_0', 'Rk')]
```
we need to merge the header columns to allow for this type of indexing 
```python 
sac_2017_df['Rk']
```

Lets do a quick magic wave of the hand and merge these headers together

Before:

In [57]:
sac_2017_df.columns[5:15]

MultiIndex([('Unnamed: 5_level_0', 'W/L'),
            ('Unnamed: 6_level_0',  'Tm'),
            ('Unnamed: 7_level_0', 'Opp'),
            (              'Team',  'FG'),
            (              'Team', 'FGA'),
            (              'Team', 'FG%'),
            (              'Team',  '3P'),
            (              'Team', '3PA'),
            (              'Team', '3P%'),
            (              'Team',  'FT')],
           )

In [51]:
merged_columns = sac_2017_df.columns.map('.'.join)

![merging](https://media.giphy.com/media/NmerZ36iBkmKk/giphy.gif)

After:

In [56]:
#collapse-hide
merged_columns[5:15]

Index(['Unnamed: 5_level_0.W/L', 'Unnamed: 6_level_0.Tm',
       'Unnamed: 7_level_0.Opp', 'Team.FG', 'Team.FGA', 'Team.FG%', 'Team.3P',
       'Team.3PA', 'Team.3P%', 'Team.FT'],
      dtype='object')

> Note: Lets break that piece of code above down for a sec: <b>sac_2017_df.columns.map('.'.join)</b> is calling the [str.join()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.join.html) function where the str is '.' for each column with the [.map()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) function

Now with the columns merged, we can keep the prefixed descriptions such as Team and Opponent so we know whose stats we're viewing but prefixes like 'Unnamed: 0_level_0' are no use to us. 

We can use regular expressions to remove the unneeded text in some of our column names

In [13]:
sac_2017_df.columns = merged_columns.str.replace(r"Unnamed:\ [0-9]_level_[0-9].", '', regex=True)

In [15]:
sac_2017_df.iloc[0:5, 0:15]

,Rk,G,Date,Unnamed: 3_level_1,Opp,W/L,Tm,Opp,Team.FG,Team.FGA,Team.FG%,Team.3P,Team.3PA,Team.3P%,Team.FT
0,1,1,2017-10-18,NaN,HOU,L,100,105,42,88,0.477,8,23,0.348,8
1,2,2,2017-10-20,@,DAL,W,93,88,37,87,0.425,10,23,0.435,9
2,3,3,2017-10-21,@,DEN,L,79,96,31,85,0.365,8,22,0.364,9
3,4,4,2017-10-23,@,PHO,L,115,117,43,99,0.434,9,22,0.409,20
4,5,5,2017-10-26,NaN,NOP,L,106,114,38,81,0.469,7,20,0.350,23


There is still an 'Unnmaed: 3_level_1' tag after the regex processing which represents if the team of interest was playing home or away. We won't even be using this column as is so we can just process our new column and drop 'Unnamed: 3_level_1' after.

The existing column consists of discreet values 'NaN' or @ indication if the team was playing at home or away for this instance. We can simply check if the row value is NaN using the .isnull() function in pandas and set those values as a new column

In [68]:
sac_2017_df['playing_home'] = sac_2017_df['Unnamed: 3_level_1'].isnull()

Now that we have our column we can simply drop the existing "Unnamed: 3_level_1" column because "playing_home" represents the same thing now but with true and false values

In [71]:
sac_2017_df.drop(columns=['Unnamed: 3_level_1'], inplace=True)

In [16]:
sac_2017_df.iloc[0:5, 0:15]

,Rk,G,Date,Unnamed: 3_level_1,Opp,W/L,Tm,Opp,Team.FG,Team.FGA,Team.FG%,Team.3P,Team.3PA,Team.3P%,Team.FT
0,1,1,2017-10-18,NaN,HOU,L,100,105,42,88,0.477,8,23,0.348,8
1,2,2,2017-10-20,@,DAL,W,93,88,37,87,0.425,10,23,0.435,9
2,3,3,2017-10-21,@,DEN,L,79,96,31,85,0.365,8,22,0.364,9
3,4,4,2017-10-23,@,PHO,L,115,117,43,99,0.434,9,22,0.409,20
4,5,5,2017-10-26,NaN,NOP,L,106,114,38,81,0.469,7,20,0.350,23


In order to prepare this data for a logistic regression model, we will also need to convert the non-numeric columns we plan to use to numerical values. Specifically converting the column of interest "W/L" to a numeric representation

In [19]:
sac_2017_df['dub'] = sac_2017_df['W/L'] == 'W'

True values in this new column represent the team of interest got the dub or the Wu as Mastah Killah would say #WuTang #ATLUnited
![Wu](https://media.giphy.com/media/2sceLbzj36eSxvcsYo/giphy.gif)

In [22]:
sac_2017_df.iloc[0:5, -10:]

,Opponent.FTA,Opponent.FT%,Opponent.ORB,Opponent.TRB,Opponent.AST,Opponent.STL,Opponent.BLK,Opponent.TOV,Opponent.PF,dub
0,29,0.931,12,44,19,7,3,14,14,False
1,21,0.714,7,36,19,8,7,12,13,True
2,20,0.600,18,58,25,7,2,16,19,False
3,27,0.852,6,45,20,6,5,20,25,False
4,23,0.739,10,47,22,5,3,15,24,False


### Might as well make a pipeline
We have established, at least, our first pass at preparing the dataset. Since we will have to prepare the other dataframes in a similar way we can mitigate this by creating a data pipeline. This pipeline will take each original dataframe in and run the same preprocessing steps. This ensures everything is going through the same steps. Pipelines are not required but it will help you to stay organized 

To make a pipeline we'll need to make the previous steps we created into a function to pass each dataframe through

In [89]:
def data_pipeline(df):
    test = df.columns.map('.'.join).str.strip('.')
    df.columns = test.str.replace(r"Unnamed:\ [0-9]_level_[0-9].", '', regex=True)
    df['playing_home'] = df['Unnamed: 3_level_1'].isnull()
    df.drop(columns=['Unnamed: 3_level_1'], inplace=True)
    df['dub'] = df['W/L'] == 'W'
    df.drop(columns=['W/L'], inplace=True)
    return df

### Running the pipeline
We can consolidate the number of duplicate lines to run into a function and process all similar datasets with the same function. The code below reads in each dataset and immediately uses the pandas .pipe() function passing in the preprocessing function. Though we didn't use it, the [pandas.DataFrame.pipe()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pipe.html?highlight=pipe#pandas.DataFrame.pipe) function allows positional and keyword arguments to be passed in with the function to run. 

In [90]:
sac_2017_df = pd.read_csv(sac_2017_szn, header=[0, 1]).pipe(data_pipeline)
sac_2018_df = pd.read_csv(sac_2018_szn, header=[0, 1]).pipe(data_pipeline)
mil_2017_df = pd.read_csv(mil_2017_szn, header=[0, 1]).pipe(data_pipeline)
mil_2018_df = pd.read_csv(mil_2018_szn, header=[0, 1]).pipe(data_pipeline)

### The Jist
Data cleaning and preprocessing is not the most fun job in data science but it sets the foundation for whatever model that will be used. There are a number of automated tools and software that claim to automatically clean datasets but from this notebook it's easy to see it isn't a cookie cutter process. This dataset was cleaned knowing a logistic regression model was going to be used and the tags to fit the model but various other methods could have been used to clean this data. Such as scaling the dataset or one-hot encoding all string columns (but I plan to not use most of them so I didn't bother). An experienced engineer once told me coding should be the easy part. That statement didn't hit me at first but I now understand that statement speaks to the importance of understanding what you want to do with the data or a model. Speaking of model, in the next post we'll get right into logistic regression models and how to measure their success cause
![obama](https://media.giphy.com/media/C7vI9SlliHtp6o478J/giphy.gif)